# 1. Install and Import Dependencies

In [ ]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

In [ ]:
%pip install stable-baselines3[extra] protobuf==3.20.*

In [ ]:
%pip install mss pydirectinput pytesseract

In [ ]:

%pip install gymnasium

In [4]:
from mss import mss
import pydirectinput
import cv2
import numpy as np
import pytesseract
from matplotlib import pyplot as plt
import time
from gymnasium import Env
from gymnasium.spaces import Box, Discrete

# 2. Build the Environment

## 2.1 Create Environment

In [5]:
class TerrEnv(Env):
    def __init__(self):
        super().__init__()
        # Setup spaces
        self.observation_space = Box(low=0, high=255, shape=(1,1080,1920), dtype=np.uint8)
        self.action_space = Discrete(7)
        # Capture game frames
        self.cap = mss()
        self.game_location = {'top': 0, 'left': 0, 'width': 1920, 'height': 1080}
        self.done_location = {'top': 460, 'left': 760, 'width': 400, 'height': 70}
        self.action_map = {
            0: 'no_op',
            1: 'space',
            2: 'w', 
            3: 's', 
            4: 'd', 
            5: 'a', 
            6: 'h', # heal
            7: 'select sword',
            8: 'select pickaxe',
            9: 'select axe',
            10: 'select bow',
            11: 'select torch',
            12: 'select rope',
            13: 'aim',
            14: 'use tool',
            15: 'Esc', # open inventory
        }
        
        
    def step(self, action):
        
        if action == 14:
            pydirectinput.click()
        elif action == 7:
            pydirectinput.press(self.action_map[1])
        elif action == 8:
            pydirectinput.press(self.action_map[2])
        elif action == 9:
            pydirectinput.press(self.action_map[3])
        elif action == 10:
            pydirectinput.press(self.action_map[4])
        elif action == 11:
            pydirectinput.press(self.action_map[5])
        elif action == 12:
            pydirectinput.press(self.action_map[6])
        elif action == 13: # Aim enemy
            pydirectinput.press(self.action_map[action])
    
        elif action != 0:
            pydirectinput.press(self.action_map[action])

        done, _ = self.get_done() 
        observation = self.get_observation()
        # calculate real reward
        reward = 1 
        info = {}
        return observation, reward, done, info
        
    
    def reset(self):
        time.sleep(10)
        pydirectinput.click(x=150, y=150)
        return self.get_observation()
        
    def render(self):
        cv2.imshow('Game', self.current_frame)
        key = cv2.waitKey(1)
        if key == ord('q'):
            self.close()
         
    def close(self):
        cv2.destroyAllWindows()
    
    def get_observation(self):
        raw = np.array(self.cap.grab(self.game_location))[:,:,:3].astype(np.uint8)
        gray = cv2.cvtColor(raw, cv2.COLOR_BGR2GRAY)
        resized = cv2.resize(gray, (1920,1080))
        channel = np.reshape(resized, (1,1080,1920))
        return channel
    
    def get_done(self):
        done_cap = np.array(self.cap.grab(self.done_location))
        done_strings = ['You', 'You ']
        done = False
        res = pytesseract.image_to_string(done_cap)[:4]
        if res in done_strings:
            done = True
        return done, done_cap

# 2.2 Test Environment

In [6]:
env = TerrEnv()

In [14]:
obs=env.get_observation()

In [15]:
plt.imshow(cv2.cvtColor(obs[0], cv2.COLOR_GRAY2BGR))

In [18]:
done, done_cap = env.get_done()

In [40]:
plt.imshow(done_cap)

In [32]:
pytesseract.image_to_string(done_cap)[:4]

'You '

In [33]:
done

True

In [ ]:
for episode in range(10): 
    obs = env.reset()
    done = False  
    total_reward   = 0
    while not done:
        action = env.action_space.sample()
        print(action)
        obs, reward,  done, info =  env.step(action)
        total_reward  += reward
    print('Total Reward for episode {} is {}'.format(episode, total_reward))    

# 3. Train the Model

## 3.1 Create Callback

In [55]:
# Import os for file path management
import os 
# Import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback
# Check Environment    
from stable_baselines3.common import env_checker

In [56]:
env_checker.check_env(env)

In [57]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [58]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [59]:
callback = TrainAndLoggingCallback(check_freq=1000, save_path=CHECKPOINT_DIR)

## 3.2 Build DQN and Train

In [60]:
from stable_baselines3 import DQN
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack

In [61]:
env = TerrEnv()

In [62]:
model = DQN('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, buffer_size=1200000, learning_starts=10)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
model.learn(total_timesteps=100000, callback=callback)

In [ ]:
model.load('train_first/best_mode l_50000') 

# 4. Test out Model

In [ ]:
for episode in range(5): 
    obs = env.reset()
    done = False
    total_reward = 0
    while not done: 
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(int(action))
        time.sleep(0.01)
        total_reward += reward
    print('Total Reward for episode {} is {}'.format(episode, total_reward))
    time.sleep(2)